# Tan Ze Xin Dylan
# 1004385

## Load the IMDB dataset and create the vocabulary

In [2]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

EMBEDDING_DIM=50
VOCAB_SIZE=20000

# Get IMDB dataset
imdb = IMDB(split='train')

# Load English tokenizer, tagger, parser and NER
tokenizer = get_tokenizer('spacy', language='en')

# build the vocab
counter = Counter()
for i, (label, line) in enumerate(imdb):
    counter.update(tokenizer(line))

ordered_dict = OrderedDict(counter.most_common()[:VOCAB_SIZE])
vocab = vocab(ordered_dict)

# insert special tokens and set default index to 'unknown'
vocab.insert_token('<PAD>', 0)
vocab.insert_token('<UNK>', 1)
vocab.set_default_index(1)

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 43.0MB/s]


## Create embedding vectors from GloVe

In [3]:
import torchtext as text

# load glove embeddings
vec = text.vocab.GloVe(name='6B', dim=50)
# create the embedding matrix, a torch tensor in the shape (num_words+1, embedding_dim)
word_emb = vec.get_vecs_by_tokens(vocab.get_itos())

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:14<00:00, 27752.50it/s]


## Build up train/test dataset

In [4]:
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
import torch
from torch.utils.data.dataset import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# transform input text and label to ids
def process_text(text):
    return vocab(tokenizer(text))

label_to_ids = {'pos':0, 'neg':1}

# preprocess a batch of raw data (string-like text/labels) into tensors
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_to_ids[_label])
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    # label must be in the same size as target
    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter, test_iter = IMDB()

# transform datasets iterator into map style so that they can be repeatedly loaded in a new epoch
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# Split ratio of 0.95 for train and 0.05 for validation
train_size = int(len(train_dataset) * 0.95)
split_train_dataset, split_valid_dataset = random_split(train_dataset, [train_size, len(train_dataset) - train_size])

train_dataloader = DataLoader(split_train_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                             shuffle=True, collate_fn=collate_batch)
# Question 1
# Write a validation dataloader by spliting the training data
val_dataloader = DataLoader(split_valid_dataset, batch_size=128,
                            shuffle=True, collate_fn=collate_batch)

## Define the logistic regression model

In [9]:
# logistic model
import torch
import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        
        # Question 2 : Replace the EmbeddingBag using PyTorch builtin functions that does the same job of computing sentence representation by taking average of the word embeddings.
        self.embedding = nn.Embedding(*(word_vec.size())).from_pretrained(word_vec, freeze=False)

        # Question 3 : Write a Fully Connected Layer (FC layer) with output size of 100 followed by a non-linear activation e.g., ReLU

        self.layers = nn.Sequential(
            nn.Linear(embed_dim, 100), # FC1
            nn.ReLU(),
            nn.Linear(100, 1) # FC2
        )

        self._init_weights(self.layers)

    def _init_weights(self, m):
        """Initialize network parameters 
        """
        initrange = 0.5

        self.embedding.weight.data.uniform_(-initrange, initrange)

        for i in range(len(m)):
          if type(m[i]) == nn.Linear:
            m[i].weight.data.uniform_(-initrange, initrange)
            m[i].bias.data.zero_()

    def forward(self, text, offsets):

      embedded = self.embedding(text)

      output = []

      # Embedding bag gives an output of 128 by 50, thus we need to make the outputs given by Embedding the same as in Embedding bag. 
      # To do so, we need to make use of the offsets used by Embedding bag to manually get the mean of the columns for a group of rows 
      # 
      # If Embedded gives: 
      #  [[1,2],
      #   [3,4],
      #   [5,6],
      #   [7,8]] 
      # and offsets are [0,2], this means that the mean of columns should be calculated for row 0 and row 1, and row 2 and row 3
      #
      # Therefore the resulting mean tensor should be:
      #  [[(1+3)/2,(2+4)/2],
      #   [(5+7)/2,(6+8)/2]]

      for i in range(len(offsets)):

        # If not last offset, then is from offset to the next offset
        if i != len(offsets)-1:
          row_offset = embedded[offsets[i]:offsets[i+1], :]

        # This is last element, thus take from last offset to end of embedded text list
        else:
          row_offset = embedded[offsets[i]:, :]

        row_mean = torch.mean(row_offset, dim=0)
        output.append(row_mean) # Append each tensor into list

      final_output = torch.stack(output, dim=0) # Convert list to a tensor with torch.stack()
 
      # Question 4: Use the new model you define in __init__()
      return torch.sigmoid(self.layers(final_output))

## Define train and test function

In [10]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text, offsets)
        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update parameters by stepping the optimizer
        optimizer.step()
        total_acc += ((predicted_label > 0.5) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Train and evaluate the model for several epochs

In [11]:
# Hyperparameters
EPOCHS = 30 # epoch
patience = 10
highest_val_acc = 0

model = LogisticRegression(word_vec=word_emb, embed_dim=EMBEDDING_DIM).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()
total_accu = None

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    
    # Question 5: Use your validation set to early stop the model. Remember to early stop when the validation accuracy does not improve for continous N number of epochs where N is a hyperparameter. Set N = 10
    val_acc = evaluate(val_dataloader)
    print('------ Validation accuracy {:8.3f}% ------\n'.format(val_acc))

    if (val_acc >= highest_val_acc):
      highest_val_acc = val_acc
      patience = 10
    
    else:
      patience -= 1

    if patience == 0:
      print("Early stopped to prevent overfitting...")
      break


| epoch   1 |    50/  186 batches | accuracy    0.506
| epoch   1 |   100/  186 batches | accuracy    0.511
| epoch   1 |   150/  186 batches | accuracy    0.508
------ Validation accuracy   59.830% ------

| epoch   2 |    50/  186 batches | accuracy    0.509
| epoch   2 |   100/  186 batches | accuracy    0.499
| epoch   2 |   150/  186 batches | accuracy    0.507
------ Validation accuracy   59.974% ------

| epoch   3 |    50/  186 batches | accuracy    0.504
| epoch   3 |   100/  186 batches | accuracy    0.517
| epoch   3 |   150/  186 batches | accuracy    0.517
------ Validation accuracy   59.806% ------

| epoch   4 |    50/  186 batches | accuracy    0.515
| epoch   4 |   100/  186 batches | accuracy    0.517
| epoch   4 |   150/  186 batches | accuracy    0.514
------ Validation accuracy   59.782% ------

| epoch   5 |    50/  186 batches | accuracy    0.514
| epoch   5 |   100/  186 batches | accuracy    0.517
| epoch   5 |   150/  186 batches | accuracy    0.526
------ Val

In [12]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.2f}%'.format(accu_test))

test accuracy    63.91%
